# Testing Moving Average Strategy (Vectorised rather that iterrows)

### Libraries + DF setup
    - This part is the same as the iterative method

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

In [12]:
#Initial DataFrame setup 
df= yf.download('^GSPC',start='2020-01-01')
df['MA100'] = df['Adj Close'].rolling(100).mean()
df['Price'] = df['Open'].shift(-1)
df.dropna(inplace = True)
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,MA100,Price
Date,,,,,,,,
2020-05-26,3004.080078,3021.719971,2988.169922,2991.770020,2991.770020,5848500000,2967.473689,3015.649902
2020-05-27,3015.649902,3036.250000,2969.750000,3036.129883,3036.129883,6398000000,2965.256487,3046.610107
2020-05-28,3046.610107,3068.669922,3023.399902,3029.729980,3029.729980,5447020000,2963.205286,3025.169922
2020-05-29,3025.169922,3049.169922,2998.610107,3044.310059,3044.310059,7297550000,2961.185586,3038.780029
2020-06-01,3038.780029,3062.179932,3031.540039,3055.729980,3055.729980,4684170000,2959.371086,3064.780029
...,...,...,...,...,...,...,...,...
2024-07-19,5543.370117,5557.500000,5497.040039,5505.000000,5505.000000,3760570000,5272.013506,5544.540039
2024-07-22,5544.540039,5570.359863,5529.040039,5564.410156,5564.410156,3375180000,5276.875806,5565.299805
2024-07-23,5565.299805,5585.339844,5550.899902,5555.740234,5555.740234,3500210000,5281.735610,5505.839844


In [ ]:
df[['Adj Close','MA100']].plot()

### Strategy Operation 
    - Find the first Signal
        o The first instance where the close was above the Moving Average
    - Assign that to a data frame
    - learnt DF maethod: .idxmax() and found .idxmin() 

In [ ]:
(df['Adj Close']>df['MA100']).idxmax() #find the index of the first instance where iclose > MA100
df.index == (df['Adj Close']>df['MA100']).idxmax() #boolean mask as an ARRAY returning true only on the first instance of close>MA100 
first_buy = pd.Series(df.index == (df['Adj Close']>df['MA100']).idxmax(), index = df.index)                  #Assign the array to a pandas series structure called First Buy and give the series the same index as the original DF
first_buy

####    Finding the Rest of the Signals 
    - using boolean mask
    - learnt calling .diff() on a boolean mask returns a mask that only shows where
      the original changes from one value to the next.
            i.e. original [T,T,T,F,F,T]
                 diff     [-,F,F,T,F,T] - = NaN as there is no previous value to compare
                                            combining first signal with this will make NaN
                                            True. To be the first buy
           interpretation [B, , ,S, ,B]  
    - learnt using | to join series together

In [ ]:
df['Adj Close']>df['MA100'] # Returns a boolean mask that is true for all the periods were close is above the MA and false for all the others

#This isn't a signal list as there would be multiple buy signals if the close stays above the MA.

(df['Adj Close']>df['MA100']).diff() #by taking the diff of the boolean mask we return a mask that has true only when the original boolean mask switches from a from one value to another. 
                                    #i.e. at a trading signal 


# making a series in df that is a boolean mask combining first buy with the rest of the trade signals
df['Trade_Signal'] = first_buy | (df['Adj Close']>df['MA100']).diff() 
df['Trade_Signal']

### Make a DF for only when a trade signal is given
        - i.e. Filter the original DF for when Trade_Signal Column = True and assign
          it to a new DF

In [ ]:
Relevant_Trades = df[first_buy | (df['Adj Close']>df['MA100']).diff()]  # Releavnt trades is a data frame of only where a trade signal is given. i.e. wheresignals column = True
Relevant_Trades

In [ ]:
len(Relevant_Trades['Price']) #since series length is odd, we are presently in a trade which must be closed at the most recent price for strategy analysis to progress

In [ ]:
if len(Relevant_Trades) % 2 != 0:           #if the length of the trade price series is odd then append the last known price
    end = df.iloc[-1:]
    Relevant_Trades = pd.concat([Relevant_Trades,end])

Relevant_Trades

### DF complete now Calculating Returns and strategy metrics

In [ ]:
Relevant_Trades[['Price','Trade_Signal']]

In [ ]:
buys = Relevant_Trades['Price'][::2] #starting at index 0 i.e. first signal, every 2nd row is a buy
sells = Relevant_Trades['Price'][1::2] #starting at index 1 i.e. 2nd signal, every 2nd row is a buy
Returns = (sells.values -buys.values)/buys.values #because buy and sell series don't have the same index: i.e. you dont always buy and sell on the smae day, you have to subtract using .values
Returns

In [ ]:
# find strategy return
Cumulated_Returns = (Returns +1).cumprod()-1
Cumulated_Returns